In [0]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def create_model(optimizer):
  
  model = Sequential()

  embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
  model.add(embedding_layer)

  model.add(Conv1D(128, 5, activation='relu'))
  model.add(GlobalMaxPooling1D())
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer=optimizer, loss='binary_crossentropy',  metrics=['acc',f1_m,precision_m, recall_m] )
  
  return model

In [0]:
import numpy as np
from datetime import datetime

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, mean_squared_error

from smac.configspace import ConfigurationSpace
from ConfigSpace.hyperparameters import CategoricalHyperparameter, UniformFloatHyperparameter, UniformIntegerHyperparameter
from ConfigSpace.conditions import InCondition

from smac.tae.execute_func import ExecuteTAFuncDict
from smac.scenario.scenario import Scenario
from smac.facade.smac_hpo_facade import SMAC4HPO


class SMAC:

    def __init__(self, params, x_train, y_train, x_test, y_test):
        self.params = params
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
        self.result = {}

    def svm_from_config(self, config):
        
     
        config = {k : config[k] for k in config if config[k]}
        
        
        model = create_model(config['optimizer'])
        
        test = model.fit(self.x_train, self.y_train, batch_size=128, epochs=6)

        #scores = cross_val_score(model, self.x_train, self.y_train, cv=5,scoring="accuracy")

        return 1-np.mean(test.history['acc'])   # Minimize!   

    def optimize(self):
        
        cs = ConfigurationSpace()

        if 'optimizer' in self.params:

            optimizer = CategoricalHyperparameter('optimizer', self.params['optimizer'])

            cs.add_hyperparameter(optimizer)
        

        scenario = Scenario({"run_obj": "quality", "runcount-limit": 5, "cutoff-time":10, "cs": cs, "deterministic": "true"})

        print(cs.get_default_configuration())

        def_value = self.svm_from_config(cs.get_default_configuration())

        smac = SMAC4HPO(scenario=scenario, rng=np.random.RandomState(42),tae_runner=self.svm_from_config)

        incumbent = smac.optimize()

        __params = incumbent.get_dictionary()

        inc_value = self.svm_from_config(incumbent)
        
        print(__params)

        model = create_model(**__params)

        hist = model.fit(self.x_train, self.y_train, batch_size=128, epochs=6)

        time = smac.stats.wallclock_time_used
  
        loss, accuracy, f1_score, precision, recall = model.evaluate(self.x_test, self.y_test, verbose=0)
        
        del smac

        self.result = {'accuracy': accuracy,'time':time,'best_params':__params}

In [0]:
# How I call the SMAC class from above:

smac = SMAC(parameters,X_train, y_train, X_test, y_test)
smac.optimize()

print(smac.result)